In [1]:
%%capture
import numpy as np
import pandas as pd
import gc

!pip install pandarallel
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

TRAIN_PATH = '/kaggle/input/competitive-data-science-predict-future-sales/' 
items = pd.read_csv(TRAIN_PATH+'items.csv')
categories = pd.read_csv(TRAIN_PATH+'item_categories.csv')
shops = pd.read_csv(TRAIN_PATH+'shops.csv')
sales = pd.read_csv(TRAIN_PATH+'sales_train.csv')
test_df = pd.read_csv(TRAIN_PATH+'/test.csv')

Create the same features as in EDA and drop outliers

In [2]:
# yakutsk = shops['shop_name'].str.contains('!Якутск')
# shops.loc[yakutsk, 'shop_name'] = shops.loc[yakutsk, 'shop_name'].str.slice(start=1)

shops['city'] = shops['shop_name'].str.split(n=1, expand=True)[0]
other = shops['city'].isin(['Цифровой', 'Интернет-магазин', 'Выездная'])
shops.loc[other, 'city'] = 'Other'
shops['city_id'] = shops['city'].factorize()[0]

categories['primary_category'] = categories['item_category_name'].str.split(' -', n=1, expand=True)[0]
categories.loc[categories['primary_category'].str.startswith('Чистые носители'), 'primary_category'] = 'Чистые носители'
categories.loc[categories['primary_category'].str.startswith('Карты оплаты'), 'primary_category'] = 'Карты оплаты'
categories['primary_category_id'] = categories['primary_category'].factorize()[0]

sales['date'] = pd.to_datetime(sales['date'], format='%d.%m.%Y')
sales['revenue'] = sales['item_price'] * sales['item_cnt_day']
sales['is_weekend'] = sales['date'].dt.weekday.isin([5, 6]).astype('int')
sales.loc[sales.shop_id==11, 'shop_id'] = 10
sales.loc[sales.shop_id==0, 'shop_id'] = 57
sales.loc[sales.shop_id==1, 'shop_id'] = 58
sales.drop(index=sales[sales['item_price'] >= 40000].index, inplace=True)
sales.drop(index=sales[sales['item_price'] < 0].index, inplace=True)
sales.drop(index=sales[sales['item_cnt_day'] >= 900].index, inplace=True)

sales = (sales.merge(shops, on='shop_id')
              .merge(items, on='item_id')
              .merge(categories, on='item_category_id')
              )

test_df['date_block_num'] = 34
test_df = (test_df.merge(items[['item_id', 'item_category_id']], on='item_id')
                  .merge(categories[['item_category_id', 'primary_category_id']], on='item_category_id')
                  .merge(shops[['shop_id', 'city_id']], on='shop_id')
                  )

shops.to_csv('shops.csv', index=False)
categories.to_csv('item_categories.csv', index=False)

In [3]:
def downcast_dtypes(df, copy=False):
    '''
    Changes column types in the dataframe:             
    `float64` type to `float32`
    `int64`   type to `int32`
    '''
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype == "int64" and c != 'ID']
    df[float_cols] = df[float_cols].astype(np.float32, copy=copy)
    df[int_cols] = df[int_cols].astype(np.uint16, copy=copy)
    
    return df

Create a new dataset with a cartesian product of item_ids and shop_ids of every month starting from 12

In [4]:
%%time
import itertools

pair_cols = ['date_block_num', 'shop_id', 'item_id']

monthly_sets = sales[sales.date_block_num >= 12].groupby('date_block_num')[['shop_id', 'item_id']].agg(set).reset_index()
lists = monthly_sets.apply(lambda x: itertools.product([x.date_block_num], x.shop_id, x.item_id), axis=1).to_list()
new_pairs = pd.DataFrame(itertools.chain(*lists), columns=pair_cols, dtype=np.int32)
new_pairs = (new_pairs.merge(items[['item_id', 'item_category_id']], on='item_id')
                      .merge(categories[['item_category_id', 'primary_category_id']], on='item_category_id')
                      .merge(shops[['shop_id', 'city_id']], on='shop_id')
                      )
new_pairs = downcast_dtypes(new_pairs)

gc.collect()

CPU times: user 11.5 s, sys: 1.47 s, total: 12.9 s
Wall time: 12.9 s


0

In [5]:
from functools import reduce


def get_lags(df, lags, index_cols=None, downcast=True):
    if index_cols is None:
        index_cols = list(df.index.names)
        df = df.reset_index()
    
    lag_stats = [] 
    for lag in lags:
        lag_df = df.copy()
        lag_df['date_block_num'] = lag_df['date_block_num'] + lag
        lag_df.columns = [f'lag_{lag}_{col}' if col not in index_cols else col for col in lag_df.columns]
        lag_stats.append(lag_df)

    df_merged = reduce(lambda  left, right: pd.merge(left, right, on=index_cols), lag_stats)
    if downcast:
        df_merged = downcast_dtypes(df_merged)
    return df_merged


def create_stats(group_by_cols, agg_dict, rename_dict, df=sales, lags=[1, 2, 3, 12]):
    monthly_stats = (df.groupby(group_by_cols)
                       .agg(agg_dict)
                       .rename(columns=rename_dict, level=0)
                       )
    monthly_stats.columns = ['_'.join(x) for x in monthly_stats.columns.to_flat_index()]
    monthly_stats = get_lags(monthly_stats, lags)
    
    return monthly_stats, group_by_cols


def get_avg_sales(gb1, gb2, col_name, lags=[1, 2, 3, 12]):
    monthly_sales = sales.groupby(gb1)['item_cnt_day'].agg(item_cnt_month=sum)
    return create_stats(gb2, {'item_cnt_month': ['mean']}, {'item_cnt_month': col_name}, monthly_sales, lags)


def get_last_time_func(group_by_cols):
    sales_months = sales.groupby(group_by_cols)['date_block_num'].unique()
    
    def time_from_last_sale(row):
        key = tuple(row[c] for c in group_by_cols)
        if len(key) == 1:
            key = key[0]
        cur_date = row['date_block_num']
        if key not in sales_months:
            return np.nan, np.nan, 0
        m = sales_months[key]
        m = m[np.nonzero(m < cur_date)]
        if len(m) == 0:
            return np.nan, np.nan, 0
        return cur_date - m.max(), cur_date - m.min(), len(m)
    
    return time_from_last_sale, sales_months

In [6]:
# monthly_item_sales['days_in_month'] = monthly_item_sales['month'].dt.days_in_month
month = pd.Series(pd.to_datetime(sales['date'], format='%d.%m.%Y').dt.to_period('M').unique())
month[34] = pd.Period('2015-11')
num_weekends = np.busday_count(
    month.dt.start_time.to_numpy(dtype='datetime64[D]'),
    month.dt.end_time.to_numpy(dtype='datetime64[D]'),
    weekmask='0000011'
)

def get_weekday_ratio():
    ratio = sales.groupby('item_id')['is_weekend'].transform('cumsum') / (sales.groupby('item_id')['is_weekend'].transform('cumcount') + 1)
    ratio = ratio.groupby([sales.date_block_num, sales.item_id]).agg(lambda x: x.iloc[-1])
    return get_lags(ratio, [1]).rename(columns={'lag_1_0': 'weekend_weight'}), ['date_block_num', 'item_id']

In [7]:
from sklearn.linear_model import LinearRegression

def get_relative_price(price_of='item_id', lags=[1,2,3]):
    monthly_avg_price = sales.groupby(['date_block_num', 'item_id'])['item_price'].mean().groupby('date_block_num').mean()
    lr_price_trend = LinearRegression().fit(np.arange(0, 34).reshape(-1, 1), monthly_avg_price)
    price_trend = lr_price_trend.predict(np.arange(0, 35).reshape(-1, 1))

    price_trends = pd.DataFrame({'date_block_num': np.arange(0, 35), 'price_trend': price_trend})
    item_prices = sales.groupby(['date_block_num', price_of])['item_price'].mean().reset_index()
    item_prices = item_prices.merge(price_trends, on='date_block_num')
    
    rez_col_name = price_of + '_relative_price'
    item_prices[rez_col_name] = (item_prices['item_price'] - item_prices['price_trend']) / item_prices['price_trend']
    item_prices = get_lags(item_prices[['date_block_num', price_of, rez_col_name]], lags, index_cols=['date_block_num', price_of])

    return item_prices, ['date_block_num', price_of]

Create features by monthly aggregation on item-shop sales, and then taking their and averages by different categorical columns (shop, city, item, category, primary category, shop+category) lagged values.

Additional features are time from the last item-shop and item sale and the month number.

In [8]:
target = sales.groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].agg(item_cnt_month='sum')

In [9]:
%%time

features = [
    create_stats(['date_block_num', 'shop_id', 'item_id'], {'item_cnt_day': ['sum']}, {'item_cnt_day': 'target'}),
#     create_stats(['date_block_num', 'item_id'], {'item_price': ['mean', 'max', 'min', 'median']}, {'item_price': 'item_price'}),
    get_avg_sales(['date_block_num', 'shop_id', 'item_id'], ['date_block_num', 'item_id'], 'item_sales'),
    get_avg_sales(['date_block_num', 'shop_id', 'item_id'], ['date_block_num', 'shop_id'], 'shop_sales'),
    get_avg_sales(['date_block_num', 'shop_id', 'city_id', 'item_id'], ['date_block_num', 'city_id'], 'city_sales'),
    get_avg_sales(['date_block_num', 'shop_id', 'item_id', 'item_category_id'], ['date_block_num', 'item_category_id'], 'cat_sales'),
    get_avg_sales(['date_block_num', 'shop_id', 'item_id', 'primary_category_id'], ['date_block_num', 'primary_category_id'], 'primarycat_sales'),
    get_avg_sales(['date_block_num', 'shop_id', 'item_id', 'item_category_id'], ['date_block_num', 'shop_id', 'item_category_id'], 'shop_cat_sales'),
    get_avg_sales(['date_block_num', 'shop_id', 'item_id', 'primary_category_id'], ['date_block_num', 'shop_id', 'primary_category_id'], 'shop_primarycat_sales'),
    get_relative_price(lags=[1,2,3,12]),
    get_weekday_ratio()
]

gc.collect()

CPU times: user 17 s, sys: 1.12 s, total: 18.1 s
Wall time: 18.2 s


57

Merge features to the created pairs

In [10]:
%%time
index_cols = len(new_pairs.columns)

for stats, cols in features:
    new_pairs = new_pairs.merge(stats, on=cols, how='left')
    test_df = test_df.merge(stats, on=cols, how='left')
    del stats

new_pairs['month_num'] = (new_pairs['date_block_num'] % 12).astype(np.uint8)
test_df['month_num'] = 34%12

time_from_last_shop_item_sale, sales_months = get_last_time_func(['item_id', 'shop_id'])
item_shop_periods = ['time_from_last_shop_item_sale', 'time_from_first_shop_item_sale', 'num_months_with_shop_item_sales']
new_pairs[item_shop_periods] = new_pairs.parallel_apply(time_from_last_shop_item_sale, axis=1, result_type='expand').fillna(999).astype(np.uint8)
test_df[item_shop_periods] = test_df.parallel_apply(time_from_last_shop_item_sale, axis=1, result_type='expand').fillna(999).astype(np.uint8)
del sales_months

time_from_last_item_sale, sales_months = get_last_time_func(['item_id'])
item_periods = ['time_from_last_item_sale', 'time_from_first_item_sale', 'num_months_with_item_sales']
new_pairs[item_periods] = new_pairs.parallel_apply(time_from_last_item_sale, axis=1, result_type='expand').fillna(999).astype(np.uint8)
test_df[item_periods] = test_df.parallel_apply(time_from_last_item_sale, axis=1, result_type='expand').fillna(999).astype(np.uint8)
del sales_months

new_pairs.sort_values('date_block_num', inplace=True)

del features
gc.collect()

CPU times: user 4min 50s, sys: 14.8 s, total: 5min 5s
Wall time: 20min 14s


0

Fill NaN's and downcast types

In [11]:
# fill NA in relative prices and in weekend_weight with average by item_category or by primary_catery if still NA
rel_price_cols = list(new_pairs.columns[new_pairs.columns.str.contains('item_id_relative_price')]) + ['weekend_weight']

new_pairs[rel_price_cols] = new_pairs.groupby("item_category_id")[rel_price_cols].transform(lambda x: x.fillna(x.mean()))
new_pairs[rel_price_cols] = new_pairs.groupby("primary_category_id")[rel_price_cols].transform(lambda x: x.fillna(x.mean()))

test_df[rel_price_cols] = test_df.groupby("item_category_id")[rel_price_cols].transform(lambda x: x.fillna(x.mean()))
test_df[rel_price_cols] = test_df.groupby("primary_category_id")[rel_price_cols].transform(lambda x: x.fillna(x.mean()))

In [12]:
new_pairs['weekend_weight'] = new_pairs['weekend_weight'] * num_weekends[new_pairs.date_block_num]
test_df['weekend_weight'] = test_df['weekend_weight'] * num_weekends[34]

In [13]:
new_pairs.fillna(0, inplace=True)
new_pairs = downcast_dtypes(new_pairs)
new_pairs.reset_index(drop=True, inplace=True)  # to save current order

gc.collect()

53

In [14]:
%%time
lag2_cols = new_pairs.columns[new_pairs.columns.str.contains('lag_2_')]
lag1_cols = [c.replace('2', '1') for c in lag2_cols]
lag3_cols = [c.replace('3', '1') for c in lag2_cols]
ratio12_cols = [f'ratio_1_to_2_'+c[6:] for c in lag2_cols]
ratio23_cols = [f'ratio_2_to_3_'+c[6:] for c in lag2_cols]
mean_cols = [f'last_3m_avg_'+c[6:] for c in lag2_cols]

def create_ratios_and_avgs(df):
    lags1 = df[lag1_cols].to_numpy(dtype='float32')
    lags2 = df[lag2_cols].to_numpy(dtype='float32')
    lags3 = df[lag3_cols].to_numpy(dtype='float32')
    df[ratio12_cols] = lags1 / (lags2 + 1e-4)
    df[ratio23_cols] = lags2 / (lags3 + 1e-4)
    df[mean_cols] = (lags1 + lags2 + lags3) / 3
    del lags1, lags2, lags3
    
create_ratios_and_avgs(new_pairs)
create_ratios_and_avgs(test_df)

new_pairs = downcast_dtypes(new_pairs)
test_df = downcast_dtypes(test_df)

CPU times: user 44.2 s, sys: 11.7 s, total: 56 s
Wall time: 56 s


In [15]:
new_pairs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6424946 entries, 0 to 6424945
Data columns (total 77 columns):
 #   Column                                   Dtype  
---  ------                                   -----  
 0   date_block_num                           int32  
 1   shop_id                                  int32  
 2   item_id                                  int32  
 3   item_category_id                         uint16 
 4   primary_category_id                      uint16 
 5   city_id                                  uint16 
 6   lag_1_target_sum                         float32
 7   lag_2_target_sum                         float32
 8   lag_3_target_sum                         float32
 9   lag_12_target_sum                        float32
 10  lag_1_item_sales_mean                    float32
 11  lag_2_item_sales_mean                    float32
 12  lag_3_item_sales_mean                    float32
 13  lag_12_item_sales_mean                   float32
 14  lag_1_shop_sales_m

In [16]:
new_pairs = new_pairs.merge(target, on=['date_block_num', 'shop_id', 'item_id'], how='left')
new_pairs['item_cnt_month'] = new_pairs['item_cnt_month'].fillna(0)

In [17]:
new_pairs

,date_block_num,shop_id,item_id,item_category_id,primary_category_id,city_id,lag_1_target_sum,lag_2_target_sum,lag_3_target_sum,lag_12_target_sum,...,last_3m_avg_target_sum,last_3m_avg_item_sales_mean,last_3m_avg_shop_sales_mean,last_3m_avg_city_sales_mean,last_3m_avg_cat_sales_mean,last_3m_avg_primarycat_sales_mean,last_3m_avg_shop_cat_sales_mean,last_3m_avg_shop_primarycat_sales_mean,last_3m_avg_item_id_relative_price,item_cnt_month
0,12,2,27,19,5,1,0.0,0.0,0.0,0.0,...,0.0,1.0,2.313585,2.313585,3.221215,3.029008,2.671068,2.543380,0.865800,0.0
1,12,42,18824,40,10,19,0.0,0.0,0.0,0.0,...,0.0,0.0,2.903593,2.818689,2.143571,1.926840,2.254596,2.135393,-0.788554,0.0
2,12,42,18825,40,10,19,0.0,0.0,0.0,0.0,...,0.0,0.0,2.903593,2.818689,2.143571,1.926840,2.254596,2.135393,-0.788554,1.0
3,12,6,2628,55,12,5,0.0,0.0,0.0,0.0,...,0.0,0.0,2.469531,2.454091,1.465040,1.427053,1.523858,1.405653,-0.624069,0.0
4,12,42,18826,40,10,19,0.0,0.0,0.0,0.0,...,0.0,1.0,2.903593,2.818689,2.143571,1.926840,2.254596,2.135393,-0.448500,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6424941,33,26,13217,47,11,13,0.0,0.0,0.0,0.0,...,0.0,0.0,2.097189,2.405250,0.000000,2.079722,0.000000,1.636012,-0.557390,0.0
6424942,33,26,13240,47,11,13,0.0,0.0,0.0,0.0,...,0.0,0.0,2.097189,2.405250,0.000000,2.079722,0.000000,1.636012,-0.557390,0.0
6424943,33,26,13252,47,11,13,0.0,0.0,0.0,0.0,...,0.0,0.0,2.097189,2.405250,0.000000,2.079722,0.000000,1.636012,-0.557390,0.0
6424944,33,58,564,78,14,28,0.0,0.0,0.0,0.0,...,0.0,0.0,1.822550,1.920476,1.419540,2.271406,0.000000,2.060606,-0.619381,0.0


Parquet is probably the best option to save dataframes in terms of file size, IO speed and (maybe) the RAM consumption. But, unfortunately, it doesn't support float16.

In [18]:
result_cols = list(new_pairs.columns[1:-1])  # all the cols except date_block_num and target
target_col = [new_pairs.columns[-1]]
new_pairs[['date_block_num']].to_parquet('date_block_num.parquet')
new_pairs[result_cols].to_parquet('X_train.parquet')
new_pairs[target_col].to_parquet('y_train.parquet')

gc.collect()

12

Same for the test dataset

In [19]:
test_df.fillna(0, inplace=True)
test_df = downcast_dtypes(test_df)

test_df = test_df.set_index('ID').sort_index()  # it's important to save test set in the correct order 

In [20]:
test_df

,shop_id,item_id,date_block_num,item_category_id,primary_category_id,city_id,lag_1_target_sum,lag_2_target_sum,lag_3_target_sum,lag_12_target_sum,...,ratio_2_to_3_item_id_relative_price,last_3m_avg_target_sum,last_3m_avg_item_sales_mean,last_3m_avg_shop_sales_mean,last_3m_avg_city_sales_mean,last_3m_avg_cat_sales_mean,last_3m_avg_primarycat_sales_mean,last_3m_avg_shop_cat_sales_mean,last_3m_avg_shop_primarycat_sales_mean,last_3m_avg_item_id_relative_price
ID,,,,,,,,,,,,,,,,,,,,,
0,5,5037,34,19,5,4,0.0,0.0,0.0,0.0,...,0.998330,0.0,3.018932,1.744383,1.744383,2.092322,2.545965,2.064780,1.956308,0.261190
1,5,5320,34,55,12,4,0.0,0.0,0.0,0.0,...,1.000156,0.0,0.000000,1.744383,1.744383,1.280903,1.285787,1.143670,1.153759,-0.643005
2,5,5233,34,19,5,4,0.0,0.0,0.0,0.0,...,0.999751,0.0,0.000000,1.744383,1.744383,2.092322,2.545965,2.064780,1.956308,0.430780
3,5,5232,34,23,5,4,0.0,0.0,0.0,0.0,...,0.999569,0.0,0.000000,1.744383,1.744383,2.017877,2.545965,1.705379,1.956308,0.344241
4,5,5268,34,20,5,4,0.0,0.0,0.0,0.0,...,0.999896,0.0,0.000000,1.744383,1.744383,4.190004,2.545965,2.866667,1.956308,1.037235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214195,45,18454,34,55,12,20,0.0,0.0,0.0,0.0,...,1.000156,0.0,0.000000,1.565560,1.506956,1.280903,1.285787,1.143247,1.211439,-0.643005
214196,45,16188,34,64,13,20,0.0,0.0,0.0,0.0,...,0.999770,0.0,0.000000,1.565560,1.506956,1.294423,2.455249,1.000000,1.498738,0.442010
214197,45,15757,34,55,12,20,0.0,0.0,0.0,0.0,...,1.000135,0.0,1.000000,1.565560,1.506956,1.280903,1.285787,1.143247,1.211439,-0.743396


In [21]:
test_df[result_cols].to_parquet('X_test.parquet')